# 处理缺失值

- 大多数教程里使用的数据与现实工作中的数据的区别在于后者很少是干净整齐的，许多目前流行的数据集都会有数据缺失的现象。更为甚者，处理不同数据源缺失值的方法还不同。
- 我们将在本节介绍一些处理缺失值的通用规则，Pandas对缺失值的表现形式，并演示Pandas自带的几个处理缺失值的工具的用法。本节以及全书涉及的缺失值主要有三种形式：null、NaN或NA。

### 选择处理缺失值的方法

- 在数据表或DataFrame中有很多识别缺失值的方法。一般情况下可以分为两种：一种方法是通过一个覆盖全局的掩码表示缺失值，另一种方法是用一个标签值（sentinel value）表示缺失值。
- 在掩码方法中，掩码可能是一个与原数组维度相同的完整布尔类型数组，也可能是用一个比特（0或1）表示有缺失值的局部状态。
- 在标签方法中，标签值可能是具体的数据（例如用 -9999表示缺失的整数），也可能是些极少出现的形式。另外，标签值还可能是更全局的值，比如用NaN（不是一个数）表示缺失的浮点数，它是IEEE浮点数规范中指定的特殊字符。
- 使用这两种方法之前都需要先综合考量：使用单独的掩码数组会额外出现一个布尔类型数组，从而增加存储与计算的负担；而标签值方法缩小了可以被表示为有效值的范围，可能需要在CPU或GPU算术逻辑单元中增加额外的（往往也不是最优的）计算逻辑。通常使用的NaN也不能表示所有数据类型。
- 大多数情况下，都不存在最佳选择，不同的编程语言与系统使用不同的方法。例如，R语言在每种数据类型中保留一个比特作为缺失数据的标签值，而SciDB系统会在每个单元后面加一个额外的字节表示NA状态。

## Pandas的缺失值

- Pandas里处理缺失值的方式延续了NumPy程序包的方式，并没有为浮点数据类型提供内置的NA作为缺失值。
- Pandas原本也可以按照R语言采用的比特模式为每一种数据类型标注缺失值，但是这种方法非常笨拙。R语言包含4种基本数据类型，而NumPy支持的类型远超4种。例如，R语言只有一种整数类型，而NumPy支持14 种基本的整数类型，可以根据精度、符号、编码类型按需选择。如果要为NumPy的每种数据类型都设置一个比特标注缺失值，可能需要为不同类型的不同操作耗费大量的时间与精力，其工作量几乎相当于创建一个新的NumPy程序包。另外，对于一些较小的数据类型（例如8位整型数据），牺牲一个比特作为缺失值标注的掩码还会导致其数据范围缩小。
- 当然，NumPy也是支持掩码数据的，也就是说可以用一个布尔掩码数组为原数组标注“无缺失值”或“有缺失值”。Pandas也集成了这个功能，但是在存储、计算和编码维护方面都需要耗费不必要的资源，因此这种方式并不可取。
- 综合考虑各种方法的优缺点，Pandas最终选择用标签方法表示缺失值，包括两种Python原有的缺失值：浮点数据类型的NaN值，以及Python的None对象。后面我们将会发现，虽然这么做也会有一些副作用，但是在实际运用中的效果还是不错的。

### 1. None：Python对象类型的缺失值

- Pandas可以使用的第一种缺失值标签是None，它是一个Python单体对象，经常在代码中表示缺失值。由于None是一个Python对象，所以不能作为任何NumPy / Pandas数组类型的缺失值，只能用于'object'数组类型（即由Python对象构成的数组）

In [1]:
import numpy as np
import pandas as pd
vals1 = np.array([1,None,3,4])
vals1

# 这里dtype=object表示NumPy认为由于这个数组是Python对象构成的，
# 因此将其类型判断为object。虽然这种类型在某些情景中非常有用，
# 对数据的任何操作最终都会在Python层面完成，但是在进行常见的快速操作时，
# 这种类型比其他原生类型数组要消耗更多的资源

array([1, None, 3, 4], dtype=object)

In [2]:
for dtype in ["object","int"]:
    print("dtype=",dtype)
    
    %timeit np.arange(1E6,dtype=dtype).sum()
    print()

dtype= object
324 ms ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

dtype= int
13.2 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [3]:
# 使用Python对象构成的数组就意味着如果你对一个包含None的数组进行累计操作，
# 如sum()或者min()，那么通常会出现类型错误

vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### 2. NaN：数值类型的缺失值

- 另一种缺失值的标签是NaN（全称Not a Number，不是一个数字），是一种按照IEEE浮点数标准设计、在任何系统中都兼容的特殊浮点数：

In [5]:
vals2 = np.array([1,np.nan,3,4])
vals2.dtype


dtype('float64')

- 请注意，NumPy会为这个数组选择一个原生浮点类型，这意味着和之前的object类型数组不同，这个数组会被编译成C代码从而实现快速操作。你可以把NaN看作是一个数据类病毒——它会将与它接触过的数据同化。无论和NaN进行何种操作，最终结果都是NaN

In [6]:
1+np.nan

nan

In [7]:
0*np.nan

nan

In [8]:
# 虽然这些累计操作的结果定义是合理的（即不会抛出异常），但是并非总是有效的：

vals2.sum(),vals2.min(),vals2.max()

C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


(nan, nan, nan)

In [9]:
# NumPy也提供了一些特殊的累计函数，它们可以忽略缺失值的影响：

np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)
# 谨记，NaN是一种特殊的浮点数，不是整数、字符串以及其他数据类型

(8.0, 1.0, 4.0)

### Pandas中NaN与None的差异

- 虽然NaN与None各有各的用处，但是Pandas把它们看成是可以等价交换的，在适当的时候会将两者进行替换：

In [10]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [11]:
# Pandas会将没有标签值的数据类型自动转换为NA。例如，当我们将整型数组中的一个值设置为np.nan时，这个值就会强制转换成浮点数缺失值NA

x = pd.Series(range(2),dtype=int)
x

0    0
1    1
dtype: int32

In [13]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

- 请注意，除了将整型数组的缺失值强制转换为浮点数，Pandas还会自动将None转换为NaN。（需要注意的是，现在GitHub上Pandas项目中已经有人提议增加一个原生的整型NA，不过到编写本书时还尚未实现。）
- 尽管这些仿佛会魔法的类型比R语言等专用统计语言的缺失值要复杂一些，但是Pandas的标签 / 转换方法在实践中的效果非常好，在我个人的使用过程中几乎没有出过问题。

### Pandas对NA缺失值进行强制转换的规则

![002 缺省值](./media/Pandas对NA缺失值进行强制转换的规则.PNG)

- 需要注意的是，Pandas中字符串类型的数据通常是用object类型存储的

## 处理缺省值

- 我们已经知道，Pandas基本上把None和NaN看成是可以等价交换的缺失值形式。为了完成这种交换过程，Pandas提供了一些方法来发现、剔除、替换数据结构中的缺失值，主要包括以下几种。
- isnull()
  创建一个布尔类型的掩码标签缺失值。
- notnull()
  与isnull()操作相反。dropna()
  返回一个剔除缺失值的数据。
- fillna()
  返回一个填充了缺失值的数据副本。

- 本节将用简单的示例演示这些方法。


In [14]:
# 1. 发现缺失值
# Pandas数据结构有两种有效的方法可以发现缺失值：isnull()和notnull()。每种方法都返回布尔类型的掩码数据，例
import numpy as np
import pandas as pd
data = pd.Series([1,np.nan,'hello',None])
data.isnull()

# 就像在3.3节中介绍的，布尔类型掩码数组可以直接作为Series或DataFrame的索引使用

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()]

# 在Series里使用的isnull()和notnull()同样适用于DataFrame，产生的结果同样是布尔类型。


0        1
2    hello
dtype: object

In [16]:
# 2. 剔除缺失值
# 除了前面介绍的掩码方法，还有两种很好用的缺失值处理方法，分别是dropna()（剔除缺失值）和fillna()（填充缺失值）。在Series上使用这些方法非常简单：

data.dropna()

0        1
2    hello
dtype: object

In [17]:
# 而在DataFrame上使用它们时需要设置一些参数
df = pd.DataFrame([[1,np.nan,2],[2,2,5],[np.nan,4,6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,2.0,5
2,NaN,4.0,6


In [18]:
# 我们没法从DataFrame中单独剔除一个值，要么是剔除缺失值所在的整行，要么是整列。根据实际需求，有时你需要剔除整行，有时可能是整列，DataFrame中的dropna()会有一些参数可以配置。
# 默认情况下，dropna()会剔除任何包含缺失值的整行数据

df.dropna()

,0,1,2
1,2.0,2.0,5
